<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/Hierarchical_Clustering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://towardsdatascience.com/machine-learning-algorithms-part-12-hierarchical-agglomerative-clustering-example-in-python-1e18e0075019#:~:text=Looking%20at%20the%20dendrogram%2C%20the,number%20of%20clusters%20is%205.


In [0]:
!git clone -l -s https://github.com/cagBRT/Machine-Learning.git cloned-repo
%cd cloned-repo
!ls

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import numpy as np
from pandas import read_csv
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

In [0]:
path = "pima_indians_diabetes.csv"
headernames = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(path, names = headernames)

In [0]:
#!cat pima_indians_diabetes.csv


In [0]:
data["preg"] = pd.to_numeric(data[1:]["preg"], downcast="float")
data["plas"] = pd.to_numeric(data[1:]["plas"], downcast="float")
data["pres"] = pd.to_numeric(data[1:]["pres"], downcast="float")
data["skin"] = pd.to_numeric(data[1:]["skin"], downcast="float")
data["test"] = pd.to_numeric(data[1:]["test"], downcast="float")
data["age"] = pd.to_numeric(data[1:]["age"], downcast="float")

data["mass"] = pd.to_numeric(data[1:]["mass"], downcast="float")
data["pedi"] = pd.to_numeric(data[1:]["pedi"], downcast="float")
data["age"] = pd.to_numeric(data[1:]["age"], downcast="float")
data["class"] = pd.to_numeric(data[1:]["class"], downcast="float")

In [0]:
array = data.values
X = array[:,0:8]
Y = array[:,8]

In [0]:
data = data.drop([0])

In [0]:
data.shape
data.tail()

In [0]:
data.describe

In [0]:
plt.figure(figsize = (10, 7))
plt.scatter(data["preg"], data["class"])
plt.xlabel("preg")
plt.ylabel("class")

Starting from each label at the bottom, you can see a vertical line up to a horizontal line. The height of that horizontal line tells you about the distance at which this label was merged into another label or cluster. You can find that other cluster by following the other vertical line down again. If you don't encounter another horizontal line, it was just merged with the other label you reach, otherwise it was merged into another cluster that was formed earlier.

horizontal lines are cluster merges
vertical lines tell you which clusters/labels were part of merge forming that new cluster
heights of the horizontal lines tell you about the distance that needed to be "bridged" to form the new cluster

In [0]:
plt.figure(figsize = (20, 10))
plt.title("Patient Dendograms")
plt.xlabel('sample index')
plt.ylabel('distance')
dend = shc.dendrogram(shc.linkage(data, method = 'ward'))

A huge jump in distance is typically what we're interested in if we want to argue for a certain number of clusters. 

In [0]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata


In [0]:
# set cut-off to 50
max_d = 3000  # max_d as in max_distance

In [0]:
fancy_dendrogram(
    Z,
    truncate_mode='lastp',
    p=12,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,
    annotate_above=10,
    max_d=max_d,  # plot a horizontal cut-off line
)
plt.show()


In [0]:
cluster = AgglomerativeClustering(n_clusters = 2, affinity = 'euclidean', linkage = 'ward')
cluster.fit_predict(patient_data)
plt.figure(figsize = (10, 7))
plt.scatter(patient_data[:,0], patient_data[:,1], c = cluster.labels_, cmap = 'rainbow')
plt.xlabel("skin")
plt.ylabel("test")